In [190]:
import glob
import time
import pandas as pd
# from xml.dom import minidom
from lxml import etree
from nltk import ngrams
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer

[nltk_data] Downloading package punkt to /home/ebryx/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ebryx/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ebryx/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [191]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

In [192]:
from sklearn.preprocessing import MultiLabelBinarizer

In [193]:
from sklearn.model_selection import train_test_split

In [194]:
t1=time.time()
path='./dataset/blogs_train/'
all_files = glob.glob(path + "/*.xml")
li = []

for filename in all_files:
  # print("file name ", filename)
  li.append(filename)
    

t2=time.time()
print("time taken to read All csv :: ", t2-t1, " seconds ")

time taken to read All csv ::  0.03737950325012207  seconds 


In [195]:
def clean_file_names(li):
  # names_list=[]
  # for i in range(len(li)):
  name_list=li.split('/')[-1]
  return name_list

In [196]:


def get_attributes_from_name(name):
  # data_object=[]
  # for data in range(len(names_list)):
  #   dummy_var=names_list[data].split('.')
  #   dummy_object={'id':dummy_var[0], 'gender':dummy_var[1],
  #                 'age':dummy_var[2], 'category':dummy_var[3],
  #                 'horoscope':dummy_var[4]
  #                 }
  #   data_object.append(dummy_object)
  dummy_var=name.split('.')
  data_object={'id':dummy_var[0], 'gender':dummy_var[1],
                  'age':dummy_var[2], 'category':dummy_var[3],
                  'horoscope':dummy_var[4]
                }
  
  return data_object



In [197]:
parser = etree.XMLParser(encoding='ISO-8859-1', recover = True, huge_tree=True)

In [198]:
def read_xml(li):
    
    
    all_objects=[]
    for i in range(len(li)):
    # print("i will run ", i+1," times")
        name_of_file=clean_file_names(li[i])
    # print(name_of_file)
        data_object=get_attributes_from_name(name_of_file)
    # print(data_object)

        post_text='  '
        tree = etree.parse(li[i], parser=parser)
        root = tree.getroot()
        for element in root.iter():
            if element.tag == 'post':
                element_text = element.text
                post_text = post_text + element_text
        mystring = post_text.replace('\n', '').replace('\r', '').replace('\t', '')
        mystring = mystring.replace('   ', '').replace('\\','')
   


   
    # print(mystring)
    # name_of_file=clean_file_names(li[i])
    # data_object=get_attributes_from_name(name_of_file)
    



        data_object['posts']=mystring   
        all_objects.append(data_object)

    return all_objects

In [199]:
t1=time.time()
data=read_xml(li[0:])
t2=time.time()

In [200]:
stop_words = set(stopwords.words('english')) 
def remove_stop_words(posts):
    filtered_post=[]
    
    word_tokens = word_tokenize(posts) 
    filtered_sentence = []   
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w)

    filtered_post.append(filtered_sentence)
    return filtered_post

In [201]:
stop_words = set(stopwords.words('english')) 
def fun_remove_stop_words(posts):
    filtered=''
    for x in posts.split(' '):
        if x not in stop_words:
            filtered+=' '+x
    return filtered    

In [202]:
for i in range(len(data)):
    data[i]['cleaned_stop_words']=fun_remove_stop_words(data[i]['posts'])

In [203]:
def remove_dashes(data):
    for i in range(len(data)):
        data[i]['cleaned_stop_words']=data[i]['cleaned_stop_words'].replace('--', ' ')
        

In [204]:
remove_dashes(data)

In [205]:
df=pd.DataFrame(data)

In [206]:
X=df[['age','cleaned_stop_words']]

In [207]:
dummy_variable_1 = pd.get_dummies(df["category"])
X=pd.concat([X,dummy_variable_1], axis=1)

In [208]:
del dummy_variable_1

In [209]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [210]:
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /home/ebryx/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [211]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1000, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
posts = vectorizer.fit_transform(X['cleaned_stop_words']).toarray()

In [212]:
# vectorizer = CountVectorizer()
# vectorizer.fit(X['cleaned_stop_words'])
# posts = vectorizer.transform(X['cleaned_stop_words'])


In [213]:
posts

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 3, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 1, 1, ..., 4, 0, 2]])

In [214]:
# transformed_posts = np.array([row.toarray() for row in posts])

In [215]:
# transformed_posts.shape=(100,35680)

In [216]:
transformed_posts=pd.DataFrame(posts)

In [217]:
X=pd.concat([X,transformed_posts],axis=1)

In [218]:
del transformed_posts

In [219]:
X=X.drop(['cleaned_stop_words'], axis=1)

In [220]:
X

,age,Accounting,Advertising,Agriculture,Architecture,Arts,Automotive,Banking,Biotech,BusinessServices,...,990,991,992,993,994,995,996,997,998,999
0,15,0,0,0,0,0,0,0,0,0,...,1,1,13,4,1,1,3,1,0,0
1,25,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,17,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,2,3,0,0
3,24,0,0,0,0,0,0,0,0,0,...,2,0,37,15,32,18,8,19,3,9
4,17,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,10,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14595,15,0,0,0,0,0,0,0,0,0,...,0,0,9,17,4,25,0,14,2,3
14596,15,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
14597,16,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
14598,23,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0


In [221]:
X.groupby('age').first().index

Index(['13', '14', '15', '16', '17', '23', '24', '25', '26', '27', '33', '34',
       '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46',
       '47', '48'],
      dtype='object', name='age')

In [222]:
X['age']=X['age'].astype(int)

In [223]:
from scipy import stats

In [224]:
t1=time.time()

X['age'] = stats.zscore(X['age'])
    
t2=time.time()
print("time ::: ", t2-t1 , " seconds to find z score")

time :::  0.0012717247009277344  seconds to find z score


In [225]:
X.head()

,age,Accounting,Advertising,Agriculture,Architecture,Arts,Automotive,Banking,Biotech,BusinessServices,...,990,991,992,993,994,995,996,997,998,999
0,-1.002358,0,0,0,0,0,0,0,0,0,...,1,1,13,4,1,1,3,1,0,0
1,0.252330,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-0.751421,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,2,3,0,0
3,0.126861,0,0,0,0,0,0,0,0,0,...,2,0,37,15,32,18,8,19,3,9
4,-0.751421,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,10,1,1,0


In [226]:
labels= []
for hor,gen in zip(df['horoscope'],df['gender']):
    labels.append((hor,gen))

In [227]:
mlb = MultiLabelBinarizer()
mlb.fit(labels)
classes=len(mlb.classes_)
print("classes  :: ", classes)
mlb.classes_

classes  ::  14


array(['Aquarius', 'Aries', 'Cancer', 'Capricorn', 'Gemini', 'Leo',
       'Libra', 'Pisces', 'Sagittarius', 'Scorpio', 'Taurus', 'Virgo',
       'female', 'male'], dtype=object)

In [228]:
labels = mlb.fit_transform(labels)

In [229]:
y=labels

In [230]:
trainX, testX, trainY, testY = train_test_split(X,y, test_size=0.2, random_state=42)

### NN

In [231]:
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.wrappers.scikit_learn import KerasClassifier

In [232]:


input_dim = trainX.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(20, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(10,  activation='tanh'))
# model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))

model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(classes, activation='softmax'))
# model.add(Activation('softmax'))



In [233]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 20)                20840     
_________________________________________________________________
dense_10 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_11 (Dense)             (None, 1024)              11264     
_________________________________________________________________
batch_normalization_3 (Batch (None, 1024)              4096      
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 14)                14350     
Total params: 50,760
Trainable params: 48,712
Non-trainable params: 2,048
_________________________________________________________________


In [234]:
trainX, testX, trainY, testY
classifier_nn = model.fit(trainX,trainY,
                    epochs=5,
                    verbose=True,
                    validation_data=(testX, testY),
                    batch_size=10)



Train on 11680 samples, validate on 2920 samples
Epoch 1/5
11680/11680 [==============================] - 6s 500us/step - loss: 0.3894 - acc: 0.8570 - val_loss: 0.3986 - val_acc: 0.8571
Epoch 2/5
11680/11680 [==============================] - 4s 376us/step - loss: 0.3801 - acc: 0.8576 - val_loss: 0.4653 - val_acc: 0.8573
Epoch 3/5
11680/11680 [==============================] - 4s 374us/step - loss: 0.3785 - acc: 0.8592 - val_loss: 0.4448 - val_acc: 0.8603
Epoch 4/5
11680/11680 [==============================] - 4s 373us/step - loss: 0.3754 - acc: 0.8619 - val_loss: 0.4286 - val_acc: 0.8666
Epoch 5/5
11680/11680 [==============================] - 4s 373us/step - loss: 0.3718 - acc: 0.8648 - val_loss: 0.4087 - val_acc: 0.8639


In [235]:
testX.shape

(2920, 1041)

In [236]:
testX[2].shape

(2920,)

In [237]:
prediction=model.predict(testX)

In [ ]:
for i in prediction:
    print(i)
   

In [239]:
testX.shape

(2920, 1041)

In [240]:
testX.iloc[1].shape

(1041,)

In [241]:
testX.iloc[1].dtypes

dtype('float64')

In [242]:
testX.iloc[1]

age            -1.002358
Accounting      0.000000
Advertising     0.000000
Agriculture     0.000000
Architecture    0.000000
                  ...   
995             6.000000
996             1.000000
997             1.000000
998             0.000000
999             0.000000
Name: 9354, Length: 1041, dtype: float64

In [243]:
a=np.zeros((1,1041))

In [244]:
sample_test=testX.iloc[1]

In [245]:
a[0][1]

0.0

In [246]:
for i in range(len(sample_test)):
    a[0][i]=sample_test.iloc[i]

In [247]:
a

array([[-1.00235836,  0.        ,  0.        , ...,  1.        ,
         0.        ,  0.        ]])

In [248]:
model.predict(a)

array([[0.06419008, 0.05262936, 0.01223183, 0.01569305, 0.00444315,
        0.16148403, 0.10932954, 0.03447536, 0.02507415, 0.0571387 ,
        0.02938532, 0.01112771, 0.2852075 , 0.13759014]], dtype=float32)

In [249]:
testY[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1])

In [ ]:
for i in model.predict(testX):
    print(i)

In [ ]:
for j in testY:
    print(j)

In [252]:
sample_test=testX.iloc[100]

In [253]:
for i in range(len(sample_test)):
    a[0][i]=sample_test.iloc[i]

In [254]:
model.predict(a)

array([[0.07986583, 0.06373521, 0.01586889, 0.01655503, 0.0051944 ,
        0.14997295, 0.09704115, 0.03626394, 0.02875593, 0.07191287,
        0.03143014, 0.01620697, 0.1674038 , 0.21979296]], dtype=float32)

In [255]:
testY[100]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1])

### End of NN